## Inspect data

In [1]:
!ls

data		 feast_infrastructure.ipynb  feature_store.yaml
data_sources.py  feature_services.py
entities.py	 features.py


In [2]:
!ls data/

data_df1.parquet  data_df3.parquet  target_df.parquet
data_df2.parquet  data_df4.parquet


## Create feast registry

In [3]:
!cat feature_store.yaml

project: breast_cancer
registry: data/registry.db
provider: local
online_store:
    path: data/online_store.db
entity_key_serialization_version: 2

In [4]:
!cat data_sources.py

from feast import FileSource


f_source1 = FileSource(
    name="df1_file_source",
    path="data/data_df1.parquet",
    timestamp_field="event_timestamp",
    description="A table describing the source of the first set of features",
    owner="test1@gmail.com"
)

f_source2 = FileSource(
    name="df2_file_source",
    path="data/data_df2.parquet",
    timestamp_field="event_timestamp",
    description="A table describing the source of the second set of features",
    owner="test2@gmail.com"
)

f_source3 = FileSource(
    name="df3_file_source",
    path="data/data_df3.parquet",
    timestamp_field="event_timestamp",
    description="A table describing the source of the third set of features",
    owner="test3@gmail.com"
)

f_source4 = FileSource(
    name="df4_file_source",
    path="data/data_df4.parquet",
    timestamp_field="event_timestamp",
    description="A table describing the source of the fourth set of features",
    owner="test4@gmail.com"
)

target_source = FileSource(
   

In [5]:
!cat entities.py

from feast import (
    Entity, ValueType
)


patient = Entity(
    name="patient_id",
    value_type=ValueType.INT32,
    description="The ID of the patient"
)


In [6]:
!cat features.py

# Importing dependencies
from datetime import timedelta
from feast import Field, FeatureView
from feast.types import Float32, Int32

from entities import *
from data_sources import *


df1_fv = FeatureView(
    name="df1_feature_view",
    ttl=timedelta(seconds=86400 * 7),
    entities=[patient],
    schema=[
        Field(name="mean radius", dtype=Float32),
        Field(name="mean texture", dtype=Float32),
        Field(name="mean perimeter", dtype=Float32),
        Field(name="mean area", dtype=Float32),
        Field(name="mean smoothness", dtype=Float32)
        ],    
    source=f_source1
)

df2_fv = FeatureView(
    name="df2_feature_view",
    ttl=timedelta(seconds=86400 * 7),
    entities=[patient],
    schema=[
        Field(name="mean compactness", dtype=Float32),
        Field(name="mean concavity", dtype=Float32),
        Field(name="mean concave points", dtype=Float32),
        Field(name="mean symmetry", dtype=Float32),
        Field(name="mean fractal dimension", dtype=

In [7]:
!cat feature_services.py

from feast import FeatureService

from features import *

feature_service_v1 = FeatureService(
    name="feature_v1",
    features=[df1_fv]
)

feature_service_v2 = FeatureService(
    name="feature_v2",
    features=[df1_fv, df2_fv]
)

feature_service_v3 = FeatureService(
    name="feature_v3",
    features=[df1_fv, df2_fv, df3_fv]
)

feature_service_v4 = FeatureService(
    name="feature_v4",
    features=[df1_fv, df2_fv, df3_fv, df4_fv]
)


In [8]:
!../../feast_env/bin/feast apply

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
Created entity patient_id
Created feature view df4_feature_view
Created feature view df2_feature_view
Created feature view target_feature_view
Created feature view df1_feature_view
Created feature view df3_feature_view
Created feature service feature_v1
Created feature service feature_v4
Created feature service feature_v2
Created feature service feature_v3

Created sqlite table breast_cancer_df1_feature_view
Created sqlite table breast_cancer_df2_feature_view
Created sqlite table breast_cancer_df3_feature_view
Created sqlite table breast_cancer_df4_feature_view
Created sqlite table breast_cancer_target_feature_view



## Listing entities, featureView, featureService

In [9]:
!ls data/

data_df1.parquet  data_df3.parquet  online_store.db  target_df.parquet
data_df2.parquet  data_df4.parquet  registry.db


In [10]:
!../../feast_env/bin/feast entities list

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
NAME        DESCRIPTION            TYPE
patient_id  The ID of the patient  ValueType.INT32


In [11]:
!../../feast_env/bin/feast feature-views list

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
NAME                 ENTITIES        TYPE
df4_feature_view     {'patient_id'}  FeatureView
df2_feature_view     {'patient_id'}  FeatureView
target_feature_view  {'patient_id'}  FeatureView
df1_feature_view     {'patient_id'}  FeatureView
df3_feature_view     {'patient_id'}  FeatureView


In [12]:
!../../feast_env/bin/feast feature-services list

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
NAME        FEATURES
feature_v1  df1_feature_view:mean radius, df1_feature_view:mean texture, df1_feature_view:mean perimeter, df1_feature_view:mean area, df1_feature_view:mean smoothness
feature_v4  df1_feature_view:mean radius, df1_feature_view:mean texture, df1_feature_view:mean perimeter, df1_feature_view:mean area, df1_feature_view:mean smoothness, df2_feature_view:mean compactness, df2_feature_view:mean concavity, df2_feature_view:mean concave points, df2_feature_view:mean symmetry, df2_feature_view:mean fractal dimension, df3_feature_view:radius error, df3_feature_view:texture error, df3_feature_view:perimeter error, df3_feature_view:area erro

In [13]:
!../../feast_env/bin/feast data-sources list

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
NAME                CLASS
df4_file_source     <class 'feast.infra.offline_stores.file_source.FileSource'>
df3_file_source     <class 'feast.infra.offline_stores.file_source.FileSource'>
df1_file_source     <class 'feast.infra.offline_stores.file_source.FileSource'>
df2_file_source     <class 'feast.infra.offline_stores.file_source.FileSource'>
target_file_source  <class 'feast.infra.offline_stores.file_source.FileSource'>


## Retrieving features

In [14]:
import pandas as pd
from feast import FeatureStore
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

In [15]:
store = FeatureStore(repo_path=".")

In [16]:
entity_df = pd.read_parquet(path="./data/target_df.parquet")
entity_df

,target,event_timestamp,patient_id
0,0,2021-05-18 22:28:59.936839,0
1,0,2021-05-19 22:28:59.936839,1
2,0,2021-05-20 22:28:59.936839,2
3,0,2021-05-21 22:28:59.936839,3
4,0,2021-05-22 22:28:59.936839,4
...,...,...,...
564,0,2022-12-03 22:28:59.936839,564
565,0,2022-12-04 22:28:59.936839,565
566,0,2022-12-05 22:28:59.936839,566
567,0,2022-12-06 22:28:59.936839,567


In [17]:
# Retrieving from the offline store with a feature service v1
feature_v1 = store.get_feature_service("feature_v1")
training_data = store.get_historical_features(features=feature_v1, entity_df=entity_df)
training_data.to_df()

,target,event_timestamp,patient_id,mean radius,mean texture,mean perimeter,mean area,mean smoothness
0,0,2021-05-18 22:28:59.936839+00:00,0,17.99,10.38,122.80,1001.0,0.11840
1,0,2021-05-19 22:28:59.936839+00:00,1,20.57,17.77,132.90,1326.0,0.08474
2,0,2021-05-20 22:28:59.936839+00:00,2,19.69,21.25,130.00,1203.0,0.10960
3,0,2021-05-21 22:28:59.936839+00:00,3,11.42,20.38,77.58,386.1,0.14250
4,0,2021-05-22 22:28:59.936839+00:00,4,20.29,14.34,135.10,1297.0,0.10030
...,...,...,...,...,...,...,...,...
564,0,2022-12-03 22:28:59.936839+00:00,564,21.56,22.39,142.00,1479.0,0.11100
565,0,2022-12-04 22:28:59.936839+00:00,565,20.13,28.25,131.20,1261.0,0.09780
566,0,2022-12-05 22:28:59.936839+00:00,566,16.60,28.08,108.30,858.1,0.08455
567,0,2022-12-06 22:28:59.936839+00:00,567,20.60,29.33,140.10,1265.0,0.11780


In [18]:
# Retrieving from the offline store with a feature service v4
feature_v4 = store.get_feature_service("feature_v4")
training_data = store.get_historical_features(features=feature_v4, entity_df=entity_df)
training_data.to_df()

,target,event_timestamp,patient_id,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,0,2021-05-18 22:28:59.936839+00:00,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,0,2021-05-19 22:28:59.936839+00:00,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,0,2021-05-20 22:28:59.936839+00:00,2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,0,2021-05-21 22:28:59.936839+00:00,3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,0,2021-05-22 22:28:59.936839+00:00,4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0,2022-12-03 22:28:59.936839+00:00,564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,0,2022-12-04 22:28:59.936839+00:00,565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,0,2022-12-05 22:28:59.936839+00:00,566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,0,2022-12-06 22:28:59.936839+00:00,567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


## Save local dataset

In [19]:
dataset = store.create_saved_dataset(
    from_=training_data,
    name="breast_cancer_dataset",
    storage=SavedDatasetFileStorage("data/breast_cancer_dataset.parquet")
)

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/feast/feature_store.py:1182: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [20]:
!ls data/

breast_cancer_dataset.parquet  data_df3.parquet  registry.db
data_df1.parquet	       data_df4.parquet  target_df.parquet
data_df2.parquet	       online_store.db


In [21]:
# Retrieving the saved dataset
training_df = store.get_saved_dataset(name="breast_cancer_dataset").to_df()
training_df

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/feast/feature_store.py:1235: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


,worst fractal dimension,texture error,worst texture,worst radius,worst compactness,event_timestamp,worst concavity,fractal dimension error,worst area,concave points error,...,compactness error,mean compactness,mean perimeter,mean concavity,mean area,symmetry error,mean symmetry,patient_id,mean texture,worst concave points
0,0.11890,0.9053,17.33,25.380,0.66560,2021-05-18 22:28:59.936839+00:00,0.7119,0.006193,2019.0,0.01587,...,0.04904,0.27760,122.80,0.30010,1001.0,0.03003,0.2419,0,10.38,0.2654
1,0.08902,0.7339,23.41,24.990,0.18660,2021-05-19 22:28:59.936839+00:00,0.2416,0.003532,1956.0,0.01340,...,0.01308,0.07864,132.90,0.08690,1326.0,0.01389,0.1812,1,17.77,0.1860
2,0.08758,0.7869,25.53,23.570,0.42450,2021-05-20 22:28:59.936839+00:00,0.4504,0.004571,1709.0,0.02058,...,0.04006,0.15990,130.00,0.19740,1203.0,0.02250,0.2069,2,21.25,0.2430
3,0.17300,1.1560,26.50,14.910,0.86630,2021-05-21 22:28:59.936839+00:00,0.6869,0.009208,567.7,0.01867,...,0.07458,0.28390,77.58,0.24140,386.1,0.05963,0.2597,3,20.38,0.2575
4,0.07678,0.7813,16.67,22.540,0.20500,2021-05-22 22:28:59.936839+00:00,0.4000,0.005115,1575.0,0.01885,...,0.02461,0.13280,135.10,0.19800,1297.0,0.01756,0.1809,4,14.34,0.1625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.07115,1.2560,26.40,25.450,0.21130,2022-12-03 22:28:59.936839+00:00,0.4107,0.004239,2027.0,0.02454,...,0.02891,0.11590,142.00,0.24390,1479.0,0.01114,0.1726,564,22.39,0.2216
565,0.06637,2.4630,38.25,23.690,0.19220,2022-12-04 22:28:59.936839+00:00,0.3215,0.002498,1731.0,0.01678,...,0.02423,0.10340,131.20,0.14400,1261.0,0.01898,0.1752,565,28.25,0.1628
566,0.07820,1.0750,34.12,18.980,0.30940,2022-12-05 22:28:59.936839+00:00,0.3403,0.003892,1124.0,0.01557,...,0.03731,0.10230,108.30,0.09251,858.1,0.01318,0.1590,566,28.08,0.1418
567,0.12400,1.5950,39.42,25.740,0.86810,2022-12-06 22:28:59.936839+00:00,0.9387,0.006185,1821.0,0.01664,...,0.06158,0.27700,140.10,0.35140,1265.0,0.02324,0.2397,567,29.33,0.2650


## Load dataset and training

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

labels = training_df['target']
features = training_df.drop(labels=['target', 'event_timestamp', 'patient_id'],
                            axis=1)

# sorted(df) to keep the order of feature fields
feature_fields = sorted(features)
features = features[feature_fields]

# split dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels, shuffle=False)

In [23]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((426, 30), (426,), (143, 30), (143,))

In [24]:
model = LogisticRegression(max_iter=1000, C=1e6)
model.fit(X=X_train, y=y_train)

/home/hoang/Documents/mlops-labs/feature-store/feast_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1000000.0, max_iter=1000)

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_pred=model.predict(X_train), y_true=y_train))
print(classification_report(y_pred=model.predict(X_test), y_true=y_test))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       177
           1       0.97      0.98      0.97       249

    accuracy                           0.97       426
   macro avg       0.97      0.97      0.97       426
weighted avg       0.97      0.97      0.97       426

              precision    recall  f1-score   support

           0       0.87      0.97      0.92        35
           1       0.99      0.95      0.97       108

    accuracy                           0.96       143
   macro avg       0.93      0.96      0.95       143
weighted avg       0.96      0.96      0.96       143



In [26]:
import joblib
joblib.dump(value=model, filename='../model.joblib')

['../model.joblib']

## Make online features

1. materialize: loads the latest features between two dates

`feast materialize 2021-01-01T00:00:00 2022-01-01T00:00:00`

2. materialize-incremental: loads features up to the provided end date:

`feast materialize-incremental 2022-01-01T00:00:00`

With `feast materialize-incremental`, the start time either `now - ttl` (the `ttl` that we defined in our feature views) or the time of the most recent materialization. If you've materialized features at least once, then subsequent materializations will only fetch features that weren't present in the store at the time of the previous materializations.

If you have several feature rows per entity, Feast will only load the latest values per entity key. As an example, if you have two entries on seperate days for the patient ID 100, only the latest entry will get materialized.

In [27]:
from datetime import datetime, timedelta

# Code for loading features to online store between two dates
"""store.materialize(
    end_date=datetime.now(),
    start_date=datetime.now() - timedelta(days=700))"""

# Loading the latest features after a previous materialize call or from the beginning of time
store.materialize_incremental(end_date=datetime.now())

Materializing 5 feature views to 2022-12-07 22:31:28+07:00 into the sqlite online store.

df4_feature_view from 2022-11-30 15:31:28+07:00 to 2022-12-07 22:31:28+07:00:


100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 713.76it/s]


df2_feature_view from 2022-11-30 15:31:28+07:00 to 2022-12-08 05:31:28+07:00:


100%|████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 949.77it/s]

target_feature_view from 2022-11-30 15:31:28+07:00 to 2022-12-08 05:31:28+07:00:



100%|███████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 1214.29it/s]


df1_feature_view from 2022-11-30 15:31:28+07:00 to 2022-12-08 05:31:28+07:00:


100%|███████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 1373.89it/s]


df3_feature_view from 2022-11-30 15:31:28+07:00 to 2022-12-08 05:31:28+07:00:


100%|███████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 1246.96it/s]


Because the `ttl = 86400 * 7` (7 days), Feast will only load the features from `2022-11-30 15:31:28+07:00` to `2022-12-08 05:31:28+07:00` (now) and we only have `8` data points in online store.

## Inference online

In [36]:
features = store.get_online_features(features=feature_v4,
                                     entity_rows=[{"patient_id": i} for i in range(1000)]).to_dict()

features_df = pd.DataFrame.from_dict(data=features)
features_df.dropna(inplace=True)
features_df

,patient_id,mean smoothness,mean radius,mean perimeter,mean texture,mean area,mean concave points,mean compactness,mean symmetry,mean fractal dimension,...,symmetry error,worst smoothness,worst texture,worst radius,worst compactness,worst concave points,worst concavity,fractal dimension error,worst perimeter,worst area
561,561,0.07449,11.200000,70.669998,29.370001,386.000000,0.00000,0.03558,0.1060,0.05502,...,0.01989,0.09267,38.299999,11.920000,0.05494,0.0000,0.0000,0.001773,75.190002,439.600006
562,562,0.10480,15.220000,103.400002,30.620001,716.900024,0.09429,0.20870,0.2128,0.07152,...,0.02137,0.14170,42.790001,17.520000,0.79170,0.2356,1.1700,0.006142,128.699997,915.000000
563,563,0.10990,20.920000,143.000000,25.090000,1347.000000,0.14740,0.22360,0.2149,0.06879,...,0.02057,0.14070,29.410000,24.290001,0.41860,0.2542,0.6599,0.006213,179.100006,1819.000000
564,564,0.11100,21.559999,142.000000,22.389999,1479.000000,0.13890,0.11590,0.1726,0.05623,...,0.01114,0.14100,26.400000,25.450001,0.21130,0.2216,0.4107,0.004239,166.100006,2027.000000
565,565,0.09780,20.129999,131.199997,28.250000,1261.000000,0.09791,0.10340,0.1752,0.05533,...,0.01898,0.11660,38.250000,23.690001,0.19220,0.1628,0.3215,0.002498,155.000000,1731.000000
566,566,0.08455,16.600000,108.300003,28.080000,858.099976,0.05302,0.10230,0.1590,0.05648,...,0.01318,0.11390,34.119999,18.980000,0.30940,0.1418,0.3403,0.003892,126.699997,1124.000000
567,567,0.11780,20.600000,140.100006,29.330000,1265.000000,0.15200,0.27700,0.2397,0.07016,...,0.02324,0.16500,39.419998,25.740000,0.86810,0.2650,0.9387,0.006185,184.600006,1821.000000
568,568,0.05263,7.760000,47.919998,24.540001,181.000000,0.00000,0.04362,0.1587,0.05884,...,0.02676,0.08996,30.370001,9.456000,0.06444,0.0000,0.0000,0.002783,59.160000,268.600006


As expected, we have a dataframe with 8 rows corresponding 8 data points.

In [37]:
features_df = features_df.drop("patient_id", axis=1)
features_df = features_df[feature_fields]
features_df

,area error,compactness error,concave points error,concavity error,fractal dimension error,mean area,mean compactness,mean concave points,mean concavity,mean fractal dimension,...,worst area,worst compactness,worst concave points,worst concavity,worst fractal dimension,worst perimeter,worst radius,worst smoothness,worst symmetry,worst texture
561,22.809999,0.008878,0.00000,0.00000,0.001773,386.000000,0.03558,0.00000,0.00000,0.05502,...,439.600006,0.05494,0.0000,0.0000,0.05905,75.190002,11.920000,0.09267,0.1566,38.299999
562,22.650000,0.048440,0.01608,0.07359,0.006142,716.900024,0.20870,0.09429,0.25500,0.07152,...,915.000000,0.79170,0.2356,1.1700,0.14090,128.699997,17.520000,0.14170,0.4089,42.790001
563,118.800003,0.043100,0.02624,0.07845,0.006213,1347.000000,0.22360,0.14740,0.31740,0.06879,...,1819.000000,0.41860,0.2542,0.6599,0.09873,179.100006,24.290001,0.14070,0.2929,29.410000
564,158.699997,0.028910,0.02454,0.05198,0.004239,1479.000000,0.11590,0.13890,0.24390,0.05623,...,2027.000000,0.21130,0.2216,0.4107,0.07115,166.100006,25.450001,0.14100,0.2060,26.400000
565,99.040001,0.024230,0.01678,0.03950,0.002498,1261.000000,0.10340,0.09791,0.14400,0.05533,...,1731.000000,0.19220,0.1628,0.3215,0.06637,155.000000,23.690001,0.11660,0.2572,38.250000
566,48.549999,0.037310,0.01557,0.04730,0.003892,858.099976,0.10230,0.05302,0.09251,0.05648,...,1124.000000,0.30940,0.1418,0.3403,0.07820,126.699997,18.980000,0.11390,0.2218,34.119999
567,86.220001,0.061580,0.01664,0.07117,0.006185,1265.000000,0.27700,0.15200,0.35140,0.07016,...,1821.000000,0.86810,0.2650,0.9387,0.12400,184.600006,25.740000,0.16500,0.4087,39.419998
568,19.150000,0.004660,0.00000,0.00000,0.002783,181.000000,0.04362,0.00000,0.00000,0.05884,...,268.600006,0.06444,0.0000,0.0000,0.07039,59.160000,9.456000,0.08996,0.2871,30.370001


In [38]:
model = joblib.load('../model.joblib')
predictions = model.predict(features_df)
predictions

array([1, 0, 0, 0, 0, 0, 0, 1])